In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import yfinance as yf
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.detrend import Detrender
import plotly.graph_objs as go
import joblib
import os

# Function to load stock data
def load_stock_data(symbol, start_date, end_date):
    try:
        data = yf.download(symbol, start=start_date, end=end_date)
        if data.empty:
            print(f"No data found for {symbol} in the specified date range.")
            return None
        return data
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return None

# Function to preprocess data
def preprocess_data(df):
    df = df[['Close']].reset_index()
    df.columns = ['ds', 'y']
    df['ds'] = pd.to_datetime(df['ds'])

    # Remove rows with NaN values
    df = df.dropna()

    # If there are still NaN values after dropping, interpolate
    if df['y'].isnull().any():
        df['y'] = df['y'].interpolate()

    return df

# Function to train Prophet model
def train_prophet(df):
    m = Prophet()
    m.fit(df)
    return m

# Function to predict with Prophet
def predict_prophet(model, future_dates):
    return model.predict(future_dates)

# Function to optimize LightGBM with Optuna
def optimize_lgbm(df, n_trials=100):
    def objective(trial):
        forecaster = PolynomialTrendForecaster(degree=1)
        detrender = Detrender(forecaster=forecaster)
        y_detrended = detrender.fit_transform(df['y'])

        X = df['ds'].astype(np.int64) // 10 ** 9  # Convert to Unix timestamp

        param = {
            'num_leaves': trial.suggest_int('num_leaves', 2, 256),
            'learning_rate': trial.suggest_float('learning_rate', 1e-8, 1.0, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        }

        model = LGBMRegressor(**param)
        model.fit(X.values.reshape(-1, 1), y_detrended)

        future_dates = pd.date_range(start=df['ds'].max(), periods=365, freq='D')
        future_df = pd.DataFrame({'ds': future_dates})
        X_future = future_df['ds'].astype(np.int64) // 10 ** 9
        y_detrended_pred = model.predict(X_future.values.reshape(-1, 1))
        y_pred = detrender.inverse_transform(pd.Series(y_detrended_pred))

        y_true = df['y'].iloc[-len(y_pred):]
        y_pred = y_pred.iloc[:len(y_true)]  # Ensure y_pred is a Series

        # Remove any NaN values before calculating MSE
        mask = ~np.isnan(y_true.values) & ~np.isnan(y_pred.values)
        y_true = y_true[mask]
        y_pred = y_pred[mask]

        return mean_squared_error(y_true, y_pred)

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)
    return study.best_params

# Function to train LightGBM model
def train_lgbm(df, params):
    forecaster = PolynomialTrendForecaster(degree=1)
    detrender = Detrender(forecaster=forecaster)
    y_detrended = detrender.fit_transform(df['y'])

    X = df['ds'].astype(np.int64) // 10 ** 9  # Convert to Unix timestamp
    model = LGBMRegressor(**params)
    model.fit(X.values.reshape(-1, 1), y_detrended)
    return model, detrender

# Function to predict with LightGBM
def predict_lgbm(model, detrender, future_dates):
    X_future = future_dates['ds'].astype(np.int64) // 10 ** 9
    y_detrended_pred = model.predict(X_future.values.reshape(-1, 1))
    y_pred = detrender.inverse_transform(pd.Series(y_detrended_pred))
    y_pred = y_pred.fillna(method='ffill').fillna(method='bfill')  # Handle NaN values
    return y_pred

# Function to calculate evaluation metrics
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    return mae, mse, rmse, mape, r2

# Main function
def main():
    symbol = input("Enter stock symbol (e.g., AAPL for Apple): ")
    start_date = pd.to_datetime(input("Enter start date (YYYY-MM-DD): "))
    end_date = pd.to_datetime(input("Enter end date (YYYY-MM-DD): "))
    forecast_period = int(input("Enter forecast period (years): "))
    n_trials = int(input("Enter number of optimization trials: "))

    if start_date >= end_date:
        print('End date must be after start date.')
        return

    # Load and preprocess data
    data = load_stock_data(symbol, start_date, end_date)
    if data is None:
        return
    df = preprocess_data(data)

    # Check if dataframe is empty after preprocessing
    if df.empty:
        print("No valid data available after preprocessing. Please try a different date range or stock symbol.")
        return

    # Train models
    prophet_model = train_prophet(df)

    # Check if optimized parameters exist
    params_file = f'{symbol}_lgbm_params.joblib'
    if os.path.exists(params_file):
        best_params = joblib.load(params_file)
        print('Loaded optimized parameters from file.')
    else:
        print('Optimizing LightGBM parameters...')
        best_params = optimize_lgbm(df, n_trials)
        joblib.dump(best_params, params_file)
        print('Optimization complete. Parameters saved.')

    lgbm_model, detrender = train_lgbm(df, best_params)

    # Make predictions
    future = prophet_model.make_future_dataframe(periods=forecast_period * 365)
    prophet_pred = predict_prophet(prophet_model, future)
    lgbm_pred = predict_lgbm(lgbm_model, detrender, future)

    # Combine predictions
    combined_pred = (prophet_pred['yhat'] + lgbm_pred) / 2

    # Display processed data
    print('Processed Stock Data:')
    print(df)

    # Plot predictions
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['ds'], y=df['y'], mode='lines', name='Actual'))
    fig.add_trace(go.Scatter(x=prophet_pred['ds'], y=prophet_pred['yhat'], mode='lines', name='Prophet'))
    fig.add_trace(go.Scatter(x=future['ds'], y=lgbm_pred, mode='lines', name='LGBM'))
    fig.add_trace(go.Scatter(x=future['ds'], y=combined_pred, mode='lines', name='Combined'))
    fig.update_layout(title='Stock Price Predictions', xaxis_title='Date', yaxis_title='Price')
    fig.show()

    # Calculate and display evaluation metrics
    y_true = df['y'][-len(lgbm_pred):]
    y_pred = combined_pred[:len(y_true)]
    mae, mse, rmse, mape, r2 = calculate_metrics(y_true, y_pred)

    print('Evaluation Metrics:')
    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'MAPE: {mape:.2f}%')
    print(f'R2 Score: {r2:.4f}')

if __name__ == '__main__':
    main()


Enter stock symbol (e.g., AAPL for Apple):  aapl
Enter start date (YYYY-MM-DD):  2019-09-14
Enter end date (YYYY-MM-DD):  2023-09-14
Enter forecast period (years):  1
Enter number of optimization trials:  20


[*********************100%%**********************]  1 of 1 completed
08:55:46 - cmdstanpy - INFO - Chain [1] start processing
08:55:46 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-08-26 08:55:46,340] A new study created in memory with name: no-name-7a5483e4-0db2-44ae-b3c0-288a27a651b8


Optimizing LightGBM parameters...


[W 2024-08-26 08:55:46,636] Trial 0 failed with parameters: {'num_leaves': 205, 'learning_rate': 2.635708245202095e-08, 'n_estimators': 100, 'min_child_samples': 35} because of the following error: ValueError('operands could not be broadcast together with shapes (365,) (365,2) ').
Traceback (most recent call last):
  File "C:\Users\nawaf\PycharmProjects\stockpredictionapp-codespace\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\nawaf\AppData\Local\Temp\ipykernel_32600\2223961737.py", line 80, in objective
    mask = ~np.isnan(y_true.values) & ~np.isnan(y_pred.values)
           ~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~
ValueError: operands could not be broadcast together with shapes (365,) (365,2) 
[W 2024-08-26 08:55:46,639] Trial 0 failed with value None.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 1006, number of used features: 1
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

ValueError: operands could not be broadcast together with shapes (365,) (365,2) 

[*********************100%%**********************]  1 of 1 completed
09:03:28 - cmdstanpy - INFO - Chain [1] start processing
09:03:30 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-08-26 09:03:30,658] A new study created in memory with name: no-name-2e80abf0-076c-4bee-ad34-39c0f6fd756b


Optimizing LightGBM parameters...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 5787, number of used features: 1
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further split

[W 2024-08-26 09:03:31,804] Trial 0 failed with parameters: {'num_leaves': 166, 'learning_rate': 2.7408563425666714e-08, 'n_estimators': 766, 'min_child_samples': 48} because of the following error: ValueError('operands could not be broadcast together with shapes (365,) (365,2) ').
Traceback (most recent call last):
  File "C:\Users\nawaf\PycharmProjects\stockpredictionapp-codespace\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\nawaf\AppData\Local\Temp\ipykernel_32872\4276940797.py", line 80, in objective
    mask = ~np.isnan(y_true.values) & ~np.isnan(y_pred.values)
           ~~~~~~~~~~~~~~~~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~
ValueError: operands could not be broadcast together with shapes (365,) (365,2) 
[W 2024-08-26 09:03:31,806] Trial 0 failed with value None.


ValueError: operands could not be broadcast together with shapes (365,) (365,2) 